# Test 3: Dataframe from track list

## Read from file

In [ ]:
SurfaceSourceFile = kds.SurfaceSourceFile("surface_source.mcpl", domain={"w": [0, 1]})
df = SurfaceSourceFile.get_pandas_dataframe()
del SurfaceSourceFile

## Plot histograms

In [ ]:
columns_to_plot = ["x", "y", "ln(E0/E)"]
# columns_to_plot = ["x", "y", "ln(E0/E)", "mu", "phi", "wgt"]
bin_size = 150
bins = [bin_size] * len(columns_to_plot)

plot_variables(df, columns_to_plot, nrows=2, ncols=3, bins=bins)

In [ ]:
plot_correlated_variables(df, columns_to_plot, save=True, density=True)

## Data preparation

Voy a tomar X, Y y letargia (en ese orden)

### df splitting

In [ ]:
columns_order = ["x", "y", "ln(E0/E)"]
micro_bins = [150, 150, 150]
macro_bins = [25, 25, 25]

df = df[columns_order]

# cumul_1, micro_1, macro_1 = [], [], []
# cumul_2, micro_2, macro_2 = [], [], []
# cumul_3, micro_3, macro_3 = [], [], []

### X

In [ ]:
cumul_1, micro_1, macro_1 = get_cumulatives_1(
    df[columns_order[0]], micro_bins[0], macro_bins[0]
)

In [ ]:
histo_1 = histograms(data=df[columns_order[0]], bins=micro_bins[0])
histo_1.calculate_histogram()
# histo.plot_histogram(density=True)
# plt.show()

histo_1.calculate_histogram_cumulative()
# histo.plot_cumulative_histogram()
# plt.show()

histo_1.calculate_histogram_new(macro_bins[0])
# histo_1.plot_histogram_new()
# plt.show()

### Y

In [ ]:
cumul_2, micro_2, macro_2 = get_cumulatives_2(
    df, columns_order[0], columns_order[1], macro_1, micro_bins[1], macro_bins[1]
)

In [ ]:
plt.figure()
cumulatives_2, bin_edges_2 = get_cumulatives(
    histo_1, df, column1=columns_order[0], column2=columns_order[1], plot=True
)
plt.show()

### ln(E0/E)

In [ ]:
cumul_3, micro_3, macro_3 = get_cumulatives_3(
    df, columns_order[0], columns_order[1], columns_order[2], macro_1, macro_2, micro_bins[2], macro_bins[2]
)


In [ ]:
macro_edges_2 = []
cumulatives_3 = []
edges_3 = []

for i in range(len(histo_1.counts_new)):
    df_filtered = df[
        (df[columns_order[0]] >= histo_1.bin_edges_new[i])
        & (df[columns_order[0]] <= histo_1.bin_edges_new[i + 1])
    ][columns_order[1]]

    # # Sort the data based on the y column
    # df_filtered.sort_values("y", inplace=True)
    # df_filtered.reset_index(drop=True, inplace=True)

    histo_2 = histograms(data=df_filtered, bins=micro_bins[1])
    histo_2.calculate_histogram()
    histo_2.calculate_histogram_cumulative()
    histo_2.calculate_histogram_new(macro_bins[1])
    macro_edges_2.append(histo_2.bin_edges_new)
    cum, bin_edges = get_cumulatives(
        histo_2, df, column1=columns_order[1], column2=columns_order[2], plot=False
    )

    cumulatives_3.append(cum)
    edges_3.append(bin_edges)

## Sampleo XY

In [ ]:
N = 10_000_000
sampled_values_x, sampled_values_y = sample(
    histo_x.cumulative,
    histo_x.bin_edges,
    histo_x.bin_edges_new,
    cumulatives_y,
    bin_edges_y,
    N,
)

### Plot XY to compare

In [ ]:
x_bin_edges = np.linspace(-12, 12, 150)
y_bin_edges = np.linspace(-12, 12, 150)

plot_hist2d(
    pd.Series(sampled_values_x),
    pd.Series(sampled_values_y),
    x_bin_edges,
    y_bin_edges,
    title="SAMPLED 2D Histogram of X and Y",
)
plt.show()

## Sample X, Y and u

In [ ]:
N = 10_000_000
print(N)
sampled_values_x, sampled_values_y, sampled_values_u = sample_3d(
    histo_1.cumulative,
    histo_1.bin_edges,
    histo_1.bin_edges_new,
    cumulatives_2,
    bin_edges_2,
    macro_edges_2,
    cumulatives_3,
    edges_3,
    N,
)

# sampled_values_x, sampled_values_y, sampled_values_u = sample_3d(
#     cumul_1, micro_1, macro_1, cumul_2, micro_2, macro_2, cumul_3, micro_3, N
# )

In [ ]:
# make dataframe with sampled values
df_sampled = pd.DataFrame(
    {
        columns_order[0]: sampled_values_x,
        columns_order[1]: sampled_values_y,
        columns_order[2]: sampled_values_u,
    }
)

# bin_size = 150
# bins = [bin_size] * len(columns_to_plot)

plot_correlated_variables(
    df_sampled,
    columns_order,
    save=True,
    density=True,
    filename="correlated_histograms_sampled.png",
)